In [ ]:
import shutil

shutil.rmtree('images')
shutil.rmtree('labels')

In [ ]:
# Upload zip files
from google.colab import files
import zipfile
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

# Upload files
uploaded = files.upload()

# Extract zip files
with zipfile.ZipFile('labels.zip', 'r') as zip_ref:
    zip_ref.extractall('./labels')
with zipfile.ZipFile('images.zip', 'r') as zip_ref:
    zip_ref.extractall('./images')

label_dir = './labels/labels'
image_dir = './images/images'
print(f"Image directory: {image_dir}")
print(f"Label directory: {label_dir}")

# Prepare data
data = []
for label_file in os.listdir(label_dir):
    if label_file.endswith('.txt'):
        label_path = os.path.join(label_dir, label_file)
        with open(label_path, 'r') as f:
            line = f.readline().strip()
            if line:
                parts = line.split()
                label = int(parts[0])
                filename = label_file.replace('.txt', '.jpg')
                filepath = os.path.join(image_dir, filename)
                if os.path.exists(filepath):
                    data.append({'filepath': filepath, 'label': label})

print(f"Total images found: {len(os.listdir(image_dir))}")
print(f"Valid labeled data found: {len(data)} samples.")

# Convert to DataFrame
labels_df = pd.DataFrame(data)

# Compute class weights for imbalance
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels_df['label']), y=labels_df['label'])
class_weights = {i: class_weights[i] for i in range(len(class_weights))}
print(f"Class Weights: {class_weights}")

# Train-validation split
train_df, val_df = train_test_split(labels_df, test_size=0.2, random_state=42, stratify=labels_df['label'])

# Convert labels to strings for flow_from_dataframe compatibility
train_df['label'] = train_df['label'].astype(str)
val_df['label'] = val_df['label'].astype(str)

# Image Data Generators (Enhanced)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filepath',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)
val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='filepath',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Cosine Annealing Scheduler
def cosine_annealing(epoch, lr):
    return 1e-5 + (1e-4 - 1e-5) * (1 + math.cos(math.pi * epoch / 30)) / 2

lr_scheduler = LearningRateScheduler(cosine_annealing)
checkpoint = ModelCheckpoint("best_densenet.h5", monitor='val_accuracy', save_best_only=True, mode='max')

# DenseNet121 with L2 Regularization
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=AdamW(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train frozen base
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks=[checkpoint, lr_scheduler],
    class_weight=class_weights
)

# Unfreeze top 50 layers for fine-tuning
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Re-compile for fine-tuning
model.compile(optimizer=AdamW(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Fine-tune model
history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=[checkpoint, lr_scheduler],
    class_weight=class_weights
)

# Load best weights
model.load_weights("best_densenet.h5")

# Evaluate the model
val_generator.reset()
preds = model.predict(val_generator, verbose=1)
preds = (preds > 0.5).astype(int)

print("\n Classification Report:")
print(classification_report(val_generator.classes, preds))


Saving labels.zip to labels.zip
Saving images.zip to images.zip
Image directory: ./images/images
Label directory: ./labels/labels
Total images found: 893
Valid labeled data found: 878 samples.
Class Weights: {0: np.float64(1.0477326968973748), 1: np.float64(0.9564270152505446)}
Found 702 validated image filenames belonging to 2 classes.
Found 176 validated image filenames belonging to 2 classes.
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 913ms/step - accuracy: 0.4936 - loss: 1.2093

22/22 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step - accuracy: 0.4934 - loss: 1.2092 - val_accuracy: 0.5284 - val_loss: 0.9096 - learning_rate: 1.0000e-04
Epoch 2/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 394ms/step - accuracy: 0.5003 - loss: 1.1412

22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 442ms/step - accuracy: 0.5009 - loss: 1.1407 - val_accuracy: 0.5568 - val_loss: 0.9196 - learning_rate: 9.9753e-05
Epoch 3/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 409ms/step - accuracy: 0.5092 - loss: 1.0905 - val_accuracy: 0.5455 - val_loss: 0.8925 - learning_rate: 9.9017e-05
Epoch 4/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step - accuracy: 0.4658 - loss: 1.1422

...

22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 444ms/step - accuracy: 0.7326 - loss: 0.6507 - val_accuracy: 0.6989 - val_loss: 0.6974 - learning_rate: 4.5644e-05
Epoch 19/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step - accuracy: 0.7463 - loss: 0.6217

22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 464ms/step - accuracy: 0.7458 - loss: 0.6220 - val_accuracy: 0.7102 - val_loss: 0.6945 - learning_rate: 4.1094e-05
Epoch 20/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step - accuracy: 0.7692 - loss: 0.5931

22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 446ms/step - accuracy: 0.7693 - loss: 0.5939 - val_accuracy: 0.7159 - val_loss: 0.6865 - learning_rate: 3.6697e-05
Epoch 21/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step - accuracy: 0.7192 - loss: 0.6479

22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 444ms/step - accuracy: 0.7200 - loss: 0.6472 - val_accuracy: 0.7216 - val_loss: 0.6838 - learning_rate: 3.2500e-05
Epoch 22/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 409ms/step - accuracy: 0.7422 - loss: 0.6291 - val_accuracy: 0.7102 - val_loss: 0.6808 - learning_rate: 2.8550e-05
Epoch 23/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 410ms/step - accuracy: 0.7312 - loss: 0.6366 - val_accuracy: 0.7102 - val_loss: 0.6783 - learning_rate: 2.4889e-05
Epoch 24/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 406ms/step - accuracy: 0.7313 - loss: 0.6258 - val_accuracy: 0.7159 - val_loss: 0.6812 - learning_rate: 2.1558e-05
Epoch 25/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 406ms/step - accuracy: 0.7658 - loss: 0.6006 - val_accuracy: 0.7159 - val_loss: 0.6810 - learning_rate: 1.8594e-05
Epoch 26/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 413ms/step - accuracy: 0.7635 - loss: 0.6001 - val_accuracy: 0.7159 - val_loss: 0.6811 - learning_rate: 1.6029e-05
Epoch 27/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 407ms/step - accuracy: 0.76

22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 447ms/step - accuracy: 0.7884 - loss: 0.5953 - val_accuracy: 0.7273 - val_loss: 0.6782 - learning_rate: 1.0983e-05
Epoch 30/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 409ms/step - accuracy: 0.7301 - loss: 0.6246 - val_accuracy: 0.7273 - val_loss: 0.6772 - learning_rate: 1.0247e-05
6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step

 Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.58      0.53        84
           1       0.53      0.42      0.47        92

    accuracy                           0.50       176
   macro avg       0.50      0.50      0.50       176
weighted avg       0.50      0.50      0.50       176

